# import

In [ ]:
!which python
import struct 
struct.calcsize("P")*8
# pd.set_option('display.max_rows', sub_pressure_df.shape[0]+1)

In [2]:
# !pip list
# !pip install openpyxl
#!pip install Shapely
#!pip install descartes
#!pip install hampel

In [2]:
import numpy as np
import pandas as pd
import statistics
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import rcParams
from operator import itemgetter
from typing import Callable, Dict, List, Set, Tuple
from scipy.optimize import curve_fit
import csv
import json
from collections import defaultdict
from scipy.signal import savgol_filter
import math 
import warnings
warnings.simplefilter('ignore', np.RankWarning)
from pprint import pprint
import os

In [3]:
import sys
sys.path.insert(1, '../methods')
sys.path.insert(1, '../util')
from util import SelectRows,calculate_derivative,pointInterval_to_pressure,point_dt_to_pressure,print_tuning_parameters,timeInterval_to_sub_df
from baseline_method import detect_breakpoints_startPoint, detect_breakpoints, detect_breakpoints_2, detect_breakpoints_3,detect_breakpoints_4,detect_breakpoints_startPoint2
from plot import PlotNSave
from plot2 import PlotNSave
from data_load_N_preprocess import LoadNPreprocessData
from base_classes import CurveParametersCalc
from patternRecognition_method import PatternRecognitionMethod
from tangent_method import TangentMethod
# from advanced_method import detect_max_FOD
from derivative_method import DerivativeMethod
from store_transients import StoreTransients
# from patternRecognition_method_N_tangent_method import PatternRecognition


# load data and process

## synthetic

In [4]:
#experiment setup
print("load&process Synthetic data===========================")
ground_truth_synthetic=[107, 164, 243, 328, 375, 446, 644, 713, 771, 820, 887, 953, 1036, 1101, 1179, 1254, 1319, 1396, 1456, 1513, 1598, 1648, 1719, 1784, 1875, 1938, 2017, 2073, 2185, 2247, 2315, 2369, 2433, 2487, 2552, 2610, 2835, 2916, 2965, 3044, 3101, 3240, 3284, 3368, 3411]
print("len(ground_truth_synthetic):",len(ground_truth_synthetic))
colum_names_synthetic={"pressure":{"time":"Time(hr)",
                        "measure":"Pressure (psia)",
                        "first_order_derivative":"first_order_derivative",
                        "second_order_derivative":"second_order_derivative"},
             "rate":{"time":"Time@end",
                     "measure":"Liquid rate (STB/D)"}}
data_inOneRow_synthetic=400
pressure_filePath="../data_input/Synthetic_Data.xlsx"
rate_filePath="../data_input/Synthetic_Data.xlsx"

processed_data=LoadNPreprocessData(pressure_filePath=pressure_filePath, 
                 rate_filePath=rate_filePath, 
                 colum_names=colum_names_synthetic, 
                 use_SG_smoothing=False)
pressure_df_synthetic=processed_data.pressure_df
rate_df_synthetic=processed_data.rate_df

print("len(pressure_df_synthetic)",len(pressure_df_synthetic))
print("len(rate_df_synthetic)",len(rate_df_synthetic))

pressure_measure_synthetic=list(pressure_df_synthetic[colum_names_synthetic["pressure"]["measure"]])
pressure_time_synthetic=list(pressure_df_synthetic[colum_names_synthetic["pressure"]["time"]])

load&process Synthetic data===========================
len(ground_truth_synthetic): 45
---load data from 'txt' or 'xlsx' files...
---The first & second order derivative has been calculated and appended to pressure dataframe
len(pressure_df_synthetic) 3519
len(rate_df_synthetic) 95


In [5]:
pressure_measure=pressure_measure_synthetic
pressure_time=pressure_time_synthetic
pressure_df=pressure_df_synthetic
# pressure_measure=pressure_measure_denoised[0:3500]
# pressure_time=pressure_time_denoised[0:3500]
# pressure_df=pressure_df_denoised[0:3500]
ground_truth=ground_truth_synthetic
rate_df=rate_df_synthetic
colum_names=colum_names_synthetic
data_inOneRow=data_inOneRow_synthetic

In [6]:
pressure_df.describe()

,Time(hr),Pressure (psia),first_order_derivative,second_order_derivative
count,3519.000000,3519.000000,3519.000000,3.519000e+03
mean,2116.579597,3457.894886,-71.190906,8.378750e+03
std,1396.126569,484.323488,2528.518177,2.727769e+05
min,0.000000,2000.000000,-22242.555966,-1.917049e+06
25%,998.220556,3124.711520,-135.106111,-6.595310e+02
50%,1794.649167,3434.877086,0.238346,-6.086416e-03
75%,2918.163194,3813.728698,206.369446,1.746537e+02
max,6058.868056,4654.515723,18738.272266,4.701446e+06


In [7]:
rate_df.describe()

,Time@end,Liquid rate (STB/D)
count,95.000000,95.000000
mean,2258.008205,-4104.186501
std,1447.763999,3724.516292
min,0.000000,-9882.543512
25%,1038.239444,-7579.069560
50%,1959.126944,-3727.730976
75%,3473.868333,0.000000
max,6058.868056,0.526919
